# PROBLEM 4 Yanis Tazi

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from tensorflow.keras.layers import BatchNormalization, LayerNormalization
import tensorflow as tf
from tensorflow.keras.layers import Dropout
import numpy as np
import matplotlib.pyplot as plt
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
from matplotlib import pyplot

from tensorflow.keras.datasets import cifar10  
from numpy import save
from numpy import savetxt
from numpy import loadtxt
from time import time
import pandas as pd



from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import math
import numpy as np
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from numpy import savetxt
from numpy import loadtxt
from tensorflow.keras.callbacks import Callback

Default GPU Device: /device:GPU:0


# Some functions 

In [2]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):

    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            # first layer but not first stack
            if stack > 0 and res_block == 0:  
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            # first layer but not first stack
            if stack > 0 and res_block == 0:
                # linear projection residual shortcut
                # connection to match changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


class Histories(Callback):

    def on_train_begin(self,logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# Data Preparation

In [3]:

subtract_pixel_mean = True
num_classes = 10

# load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions.
input_shape = x_train.shape[1:]

# normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# ResNet 50

# (a) P100

In [4]:
batch_size = 128 
epochs =350
data_augmentation = False



n = 8 
depth = n * 6 + 2

model_type = 'ResNet50'

model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['acc'])

Learning rate:  0.001


In [5]:



# prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate adjustment.

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)



# run training, with or without data augmentation.



In [8]:
histories = Histories()

callbacks = [checkpoint, lr_reducer, lr_scheduler,histories]

steps_per_epoch =  math.ceil(len(x_train) / batch_size)
# fit the model on the batches generated by datagen.flow().
history = model.fit(x_train,y_train,
          verbose=1,
          epochs=epochs,
          validation_data=(x_test, y_test),  ### Testing is done with original images.
          steps_per_epoch=steps_per_epoch,
          callbacks=callbacks)



Learning rate:  0.001
Epoch 1/350
391/391 [==============================] - ETA: 0s - loss: 1.8977 - acc: 0.4622
Epoch 00001: val_acc improved from -inf to 0.37320, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.001.h5
391/391 [==============================] - 25s 63ms/step - loss: 1.8977 - acc: 0.4622 - val_loss: 2.4552 - val_acc: 0.3732
Learning rate:  0.001
Epoch 2/350
390/391 [============================>.] - ETA: 0s - loss: 1.4060 - acc: 0.6257
Epoch 00002: val_acc improved from 0.37320 to 0.61560, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.002.h5
391/391 [==============================] - 23s 58ms/step - loss: 1.4056 - acc: 0.6258 - val_loss: 1.4391 - val_acc: 0.6156
Learning rate:  0.001
Epoch 3/350
390/391 [============================>.] - ETA: 0s - loss: 1.1686 - acc: 0.7075
Epoch 00003: val_acc did not improve from 0.61560
391/391 [==============================] - 22s 57ms/step - loss: 1.1687 - acc: 0.7075 - val_loss: 1.7158 - va

In [9]:
savetxt('resnet50_history_losses_p100.csv', histories.losses, delimiter=',')

# (b) k80

In [ ]:
histories = Histories()

callbacks = [checkpoint, lr_reducer, lr_scheduler,histories]

steps_per_epoch =  math.ceil(len(x_train) / batch_size)
# fit the model on the batches generated by datagen.flow().
history = model.fit(x_train,y_train,
          verbose=1,
          epochs=epochs,
          validation_data=(x_test, y_test),  ### Testing is done with original images.
          steps_per_epoch=steps_per_epoch,
          callbacks=callbacks)



Learning rate:  0.001
Epoch 1/350
391/391 [==============================] - ETA: 0s - loss: 1.8971 - acc: 0.4598
Epoch 00001: val_acc improved from -inf to 0.47490, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.001.h5
391/391 [==============================] - 81s 208ms/step - loss: 1.8971 - acc: 0.4598 - val_loss: 1.8614 - val_acc: 0.4749
Learning rate:  0.001
Epoch 2/350
391/391 [==============================] - ETA: 0s - loss: 1.3704 - acc: 0.6385
Epoch 00002: val_acc improved from 0.47490 to 0.61950, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.002.h5
391/391 [==============================] - 79s 203ms/step - loss: 1.3704 - acc: 0.6385 - val_loss: 1.4443 - val_acc: 0.6195
Learning rate:  0.001
Epoch 3/350
391/391 [==============================] - ETA: 0s - loss: 1.1506 - acc: 0.7117
Epoch 00003: val_acc did not improve from 0.61950
391/391 [==============================] - 78s 200ms/step - loss: 1.1506 - acc: 0.7117 - val_loss: 2.4275 -

In [ ]:
savetxt('resnet50_history_losses_k80.csv', histories.losses, delimiter=',')

# (c) V100

In [7]:
histories = Histories()

callbacks = [checkpoint, lr_reducer, lr_scheduler,histories]

steps_per_epoch =  math.ceil(len(x_train) / batch_size)
# fit the model on the batches generated by datagen.flow().
history = model.fit(x_train,y_train,
          verbose=1,
          epochs=epochs,
          validation_data=(x_test, y_test),  ### Testing is done with original images.
          steps_per_epoch=steps_per_epoch,
          callbacks=callbacks)



Learning rate:  0.001
Epoch 1/350
391/391 [==============================] - ETA: 0s - loss: 1.9162 - acc: 0.4635
Epoch 00001: val_acc improved from -inf to 0.41060, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.001.h5
391/391 [==============================] - 18s 45ms/step - loss: 1.9162 - acc: 0.4635 - val_loss: 2.2998 - val_acc: 0.4106
Learning rate:  0.001
Epoch 2/350
391/391 [==============================] - ETA: 0s - loss: 1.3474 - acc: 0.6474
Epoch 00002: val_acc improved from 0.41060 to 0.63590, saving model to /home/jupyter/saved_models/cifar10_ResNet50_model.002.h5
391/391 [==============================] - 16s 41ms/step - loss: 1.3474 - acc: 0.6474 - val_loss: 1.3829 - val_acc: 0.6359
Learning rate:  0.001
Epoch 3/350
391/391 [==============================] - ETA: 0s - loss: 1.1140 - acc: 0.7282
Epoch 00003: val_acc did not improve from 0.63590
391/391 [==============================] - 16s 40ms/step - loss: 1.1140 - acc: 0.7282 - val_loss: 1.7701 - va

In [ ]:
savetxt('resnet50_history_losses_v100.csv', histories.losses, delimiter=',')